In [95]:
import kagglehub as kh
import gym 
import gym_anytrading
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from gym_anytrading.envs import StocksEnv
from gym import spaces

In [96]:
path = kh.dataset_download("sudalairajkumar/cryptocurrencypricehistory")
print("Path to dataset files:", path)

Path to dataset files: /home/gabbycamm/.cache/kagglehub/datasets/sudalairajkumar/cryptocurrencypricehistory/versions/3


In [97]:
df = pd.read_csv(os.path.join(path, 'coin_Bitcoin.csv'))

In [98]:
df = df[['Open', 'High', 'Low', 'Close']]

In [99]:
df = df.astype(float)

In [100]:
df.dropna(inplace=True)

In [101]:
print(df.head())

         Open        High         Low       Close
0  134.444000  147.488007  134.000000  144.539993
1  144.000000  146.929993  134.050003  139.000000
2  139.000000  139.889999  107.720001  116.989998
3  116.379997  125.599998   92.281898  105.209999
4  106.250000  108.127998   79.099998   97.750000


In [102]:
class CryptoTradingEnv():
    #constructor: sets up trading environment
    # initializes the market data, agent's balance, trading pos, obs&act spaces
    def __init__(self, df, initial_balance=1000):
        super(CryptoTradingEnv, self).__init__()

        self.df = df
        self.current_step =0

        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0
        self.entry_price = 0

        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32
        )

    #resets environmemt at beginning of a new episode 
    def reset(self): 
        self.current_step = 0
        self.balance = self.initial_balance 
        self.position = 0 
        self.entry_price = 0
        return self._get_observation()

    #returns current state (features)
    def _get_observation(self):
        return np.array([
            self.df.iloc[self.current_step]['Close'], 
            self.df.iloc[self.current_step]['Open'],
            self.df.iloc[self.current_step]['High'], 
            self.df.iloc[self.current_step]['Low'],
            self.position
        ], dtype=np.float32)

    #executes the action and returns the new state, reward, done flag, & info
    def step(self, action):
        global reward
        
        done = False
        current_price = self.df.iloc[self.current_step]['Close']

        #execute actions
        if action ==1: # Buy
            if self.position ==0: #ENTER TRADE DO SOMETHING HERE!?!?!??!
                self.position = 1
                self.entry_price = current_price
        elif action ==2:
            if self.position ==1: # sell
                reward = current_price - self.entry_price #prof or loss
                self.balance += reward
                self.position = 0
            else:
                reward = 0
                
        else: 
            reward = 0 # Hold

        self.current_step +=1
        if self.current_step >= len(self.df)-1:
            done = True
        return self._get_observation(), reward, done, {}

        
    def render(self): 
        print(f"Step: {self.current_step}, Balance: {self.balance}, Position: {self.position}")
        
        

In [93]:
env = CryptoTradingEnv(df)

In [94]:
obs = env.reset()

for _ in range(10):
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)
    env.render()
    if done: 
        break 

Step: 1, Balance: 1000, Position: 0
Step: 2, Balance: 1000, Position: 1
Step: 3, Balance: 977.9899978637695, Position: 0
Step: 4, Balance: 977.9899978637695, Position: 1
Step: 5, Balance: 977.9899978637695, Position: 1
Step: 6, Balance: 977.9899978637695, Position: 1
Step: 7, Balance: 977.9899978637695, Position: 1
Step: 8, Balance: 977.9899978637695, Position: 1
Step: 9, Balance: 977.9899978637695, Position: 1
Step: 10, Balance: 986.3460006713867, Position: 0


In [ ]:
//exploratory data analysis remove outliers and null and find mean mode and all that